## Segmenting and Clustering Neighborhoods in Toronto City

In [3]:
import numpy as numpy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

'''Run fist with pips for dowloanding package later comment them'''
!conda install -c conda-forge geopy --yes
!pip install lxml
!pip install wikipedia
!pip install geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors
## import k-means from clustering stage
from sklearn.cluster import KMeans
import lxml
import folium # map rendering library
import wikipedia
import geocoder
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

     |████████████████████████████████| 5.5MB 4.7MB/s eta 0:00:01    |▍                               | 61kB 9.1MB/s eta 0:00:01     |███████████▉                    | 2.0MB 4.7MB/s eta 0:00:01     |█████████████████████████████   | 5.0MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 3.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 102kB 5.1MB/s ta 0:00:011
     |████████████████████████████████| 92kB 4.9MB/s eta 0:00:011
     |████████████████████████████████| 829kB 5.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Libraries imported.


### Get data from Wikipedia

In [4]:

html = wikipedia.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

df.drop(df.index[df['Borough'] == "Not assigned"], inplace = True)
#df
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x:', '.join(x)).to_frame().reset_index()

for n_list in (df["Neighborhood"]):
     if n_list == "Not assigned":
        n_list==df["'Borough'"].index(n_list)
df.head()     

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df.shape

(103, 3)

### Read Geolocation Data From Given Link

In [24]:

geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [25]:
places_with_long_lang=pd.merge(df,geo_df)
places_with_long_lang.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Visualize Data In Map

###### Create Maps For Toronto and Scarbrough

In [23]:
import geocoder # import geocoder

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude=location.latitude
longitude= location.longitude

print('The geograpical coordinate of {} are {}, {}.'.format(address,location.latitude, location.longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [26]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(places_with_long_lang['Latitude'], places_with_long_lang['Longitude'], places_with_long_lang['Borough'], places_with_long_lang['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.55,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [30]:
Scarborough_data = places_with_long_lang[places_with_long_lang['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [32]:
address ='Scarborough,Toronto,Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=17,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'BAYOAKJAHLKHPNQKFTWSUJC5OQMIWVLDPZNXDVEM4JY2S352' 
CLIENT_SECRET = '4NR5BVY1EVWON2EVK1QH0U15POEDAJXGEK3PSTNR01XE2GMN' 
VERSION = '21062020' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BAYOAKJAHLKHPNQKFTWSUJC5OQMIWVLDPZNXDVEM4JY2S352
CLIENT_SECRET:4NR5BVY1EVWON2EVK1QH0U15POEDAJXGEK3PSTNR01XE2GMN


In [33]:
# Select first building (which in our case is BedokVille):
neighborhood_name = Scarborough_data.loc[0, 'Neighborhood']
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] 
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] 
# limit of number of venues returned by Foursquare API
LIMIT = 100 
radius = 500

In [39]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
#results

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
s_venues= getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [85]:
s_venues.shape

(93, 7)

In [86]:
s_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",15,15,15,15,15,15
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",7,7,7,7,7,7
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [87]:
# one hot encoding
s_onehot = pd.get_dummies(s_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
s_onehot['Neighborhood'] = s_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [s_onehot.columns[-1]] + list(s_onehot.columns[:-1])
s_onehot = s_onehot[fixed_columns]

s_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
s_venues.shape

(93, 7)

In [112]:
s_grouped = s_onehot.groupby('Neighborhood').mean().reset_index()
s_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.0,0.000,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.0,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.000,0.000000,0.0,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.0,0.000000
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.0,0.125,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.125000,0.000000,0.000000,0.125000,0.0,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.0,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.066667,0.0,0.000000,0.0,0.066667,0.000000,0.0,0.000000,0.000000,0.0,0.066667,0.066667,0.000000,0.000000,0.066667,0.0,0.000000,0.000,0.000000,0.000000,0.066667,0.066667,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.066667,0.000000,0.000000,0.133333,0.133333,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.066667,0.0,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.142857,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000,0.000000,0.285714,0.000000,0.000000,0.000000,0.0,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.000000,0.222222,0.000000,0.0,0.000000,0.111111,0.111111,0.0,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.111111,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.0,0.

In [166]:
num_top_venues = 5

for hood in s_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = s_grouped[s_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0  General Entertainment   0.2
1           Skating Rink   0.2
2                   Café   0.2
3                   Farm   0.2
4        College Stadium   0.2


----Cedarbrae----
                venue  freq
0    Hakka Restaurant  0.12
1              Bakery  0.12
2                Bank  0.12
3     Thai Restaurant  0.12
4  Athletics & Sports  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0              Pharmacy  0.13
1           Pizza Place  0.13
2          Intersection  0.07
3  Fast Food Restaurant  0.07
4          Noodle House  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1             

In [167]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [168]:
num_top_venues = 10

import numpy as np

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = s_grouped['Neighborhood']

for ind in np.arange(s_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(s_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Convenience Store,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Skating Rink,Farm,Café,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Bakery,Bank,Fried Chicken Joint,Department Store,Discount Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Gas Station,Chinese Restaurant,Noodle House,Italian Restaurant,Intersection,Convenience Store,Bank,Shopping Mall
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Discount Store


In [169]:
# set number of clusters
kclusters = 6

clustering = s_grouped.drop('Neighborhood', 1)
clustering

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

print(kmeans)
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

s_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
s_merged = s_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

s_merged.head() # check the last columns!


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Vietnamese Restaurant,Grocery Store,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,College Stadium,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Bank,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5.0,Coffee Shop,Korean Restaurant,College Stadium,Gas Station,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Bakery,Bank,Fried Chicken Joint,Department Store,Discount Store


In [170]:
s_merged = s_merged[s_merged['Cluster Labels'].notna()]

In [174]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(s_merged['Latitude'], s_merged['Longitude'], s_merged['Neighborhood'], s_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters